In [19]:
import time
from collections import deque
import numpy as np
import serial


In [26]:
"""
Real-time IMU runner (Serial -> |Δω| -> rep FSM -> ROM proxy prints)

- NO plotting, NO file saving, NO vision
- Wearable does NO analytics (frozen)
- Uses rolling window smoothing + per-rep accumulators only
- Prints REP START / REP END / ROM_PROXY immediately

Data format (CSV per line):
SEQ,T_US,A1X,A1Y,A1Z,G1X,G1Y,G1Z,A2X,A2Y,A2Z,G2X,G2Y,G2Z
"""

import time
from collections import deque
import numpy as np
import serial

# ---------------- USER CONFIG ----------------
PORT = "COM20"       # <-- set your port
BAUD = 115200

EXPECTED_COLS = 14

# ---------- Real-time smoothing ----------
SMOOTH_WIN = 11      # more robust than 7 for live jitter (still same moving-average idea)

# ---------- Thresholds (use your validated ones as base) ----------
START_THR = 0.22
STOP_THR  = 0.16     # raised to prevent immediate stop chatter (keep ~60–75% of START_THR)

# ---------- Timing gates (robustness) ----------
START_CONFIRM_S = 0.15   # must stay above START_THR this long to start (debounce)
MIN_ACTIVE_S    = 0.30   # once started, cannot end before this age (prevents start->instant end)
MIN_REST_S      = 0.25   # must stay below STOP_THR this long to end
COOLDOWN_S      = 0.20   # after END, ignore immediate retriggers

# ---------- Post rules (still real-time prints) ----------
MIN_REP_DUR_S   = 0.60   # ignore tiny accidental reps at END (optional but strongly recommended)
MIN_ROM_DEG     = 10.0   # ignore micro reps by ROM proxy (optional but strongly recommended)

DEBUG_EVENTS = True      # prints trigger values to explain WHY start/end happened
# -------------------------------------------


def compute_dw_mag_sample(g1x, g1y, g1z, g2x, g2y, g2z) -> float:
    """|Δω| = ||ω1 - ω2||. Same math as offline."""
    w1 = np.array([g1x, g1y, g1z], dtype=np.float64)
    w2 = np.array([g2x, g2y, g2z], dtype=np.float64)
    return float(np.linalg.norm(w1 - w2))


class CausalMovingAverage:
    """Causal rolling mean (no future samples)."""
    def __init__(self, win: int):
        self.win = max(1, int(win))
        self.buf = deque(maxlen=self.win)
        self.sum = 0.0

    def update(self, x: float) -> float:
        if self.win == 1:
            return x
        if len(self.buf) == self.buf.maxlen:
            self.sum -= self.buf[0]
        self.buf.append(x)
        self.sum += x
        return self.sum / len(self.buf)


class RepStateMachine:
    IDLE = 0
    IN_REP = 1

    def __init__(self):
        self.state = self.IDLE

        self.rep_count = 0
        self.rep_start_t = None
        self.cooldown_until = -1e9

        # start/stop confirmation timers
        self.above_start_time = 0.0
        self.below_stop_time = 0.0

        # ROM proxy accumulator (radians)
        self.rom_rad = 0.0

        # last sample for trapezoid integration
        self.last_t = None
        self.last_dw_raw = None

    def update(self, t_s: float, dw_raw: float, dw_s: float):
        """Process one sample; prints events immediately."""
        if self.last_t is None:
            self.last_t = t_s
            self.last_dw_raw = dw_raw
            return

        dt = t_s - self.last_t

        # Guard against weird timestamp jumps (USB hiccups, resets, etc.)
        if dt <= 0.0 or dt > 0.25:
            self.last_t = t_s
            self.last_dw_raw = dw_raw
            # also reset confirmation timers to avoid nonsense triggers
            self.above_start_time = 0.0
            self.below_stop_time = 0.0
            return

        if self.state == self.IDLE:
            # Ignore starts during cooldown window
            if t_s < self.cooldown_until:
                self.above_start_time = 0.0
            else:
                # START debounce
                if dw_s >= START_THR:
                    self.above_start_time += dt
                else:
                    self.above_start_time = 0.0

                if self.above_start_time >= START_CONFIRM_S:
                    self.state = self.IN_REP
                    self.rep_start_t = t_s
                    self.rep_count += 1

                    # reset per-rep accumulators/timers
                    self.rom_rad = 0.0
                    self.below_stop_time = 0.0

                    # Reset last integration anchor to "now"
                    self.last_t = t_s
                    self.last_dw_raw = dw_raw

                    if DEBUG_EVENTS:
                        print(
                            f"[{t_s:8.3f}s] REP START (rep={self.rep_count}) "
                            f"dw_s={dw_s:.3f} >= {START_THR:.2f} "
                            f"confirm={self.above_start_time:.3f}s dw_raw={dw_raw:.3f}"
                        )
                    return  # important: don't fall through and integrate same sample twice

        elif self.state == self.IN_REP:
            # Per-sample ROM proxy accumulation: trapezoid integral of |Δω| (same as offline trapz)
            self.rom_rad += 0.5 * (self.last_dw_raw + dw_raw) * dt

            rep_age = t_s - (self.rep_start_t if self.rep_start_t is not None else t_s)

            # Prevent immediate END after START (fixes your "start then end instantly" issue)
            if rep_age < MIN_ACTIVE_S:
                self.below_stop_time = 0.0
            else:
                # STOP confirmation
                if dw_s <= STOP_THR:
                    self.below_stop_time += dt
                else:
                    self.below_stop_time = 0.0

                if self.below_stop_time >= MIN_REST_S:
                    rep_dur = rep_age
                    rom_deg = self.rom_rad * (180.0 / np.pi)

                    # Optional validity gates (not new metrics; just filtering junk)
                    valid = (rep_dur >= MIN_REP_DUR_S) and (rom_deg >= MIN_ROM_DEG)

                    if DEBUG_EVENTS:
                        print(
                            f"[{t_s:8.3f}s] REP END   (rep={self.rep_count}, dur={rep_dur:.2f}s) "
                            f"dw_s={dw_s:.3f} <= {STOP_THR:.2f} rest={self.below_stop_time:.3f}s "
                            f"ROM={rom_deg:.2f}deg {'VALID' if valid else 'IGNORED'}"
                        )

                    if valid:
                        print(f"            ROM_PROXY = {rom_deg:.2f} deg")
                    else:
                        print(f"            (ignored rep)")

                    # reset to idle + cooldown
                    self.state = self.IDLE
                    self.rep_start_t = None
                    self.cooldown_until = t_s + COOLDOWN_S
                    self.above_start_time = 0.0
                    self.below_stop_time = 0.0
                    self.rom_rad = 0.0

        # Update last sample memory
        self.last_t = t_s
        self.last_dw_raw = dw_raw


def parse_line(line: str):
    """
    Returns (t_s, g1x,g1y,g1z,g2x,g2y,g2z) or None.
    """
    parts = line.strip().split(",")
    if len(parts) != EXPECTED_COLS:
        return None

    try:
        # seq = int(parts[0])  # available if you want drop detection later
        t_us = int(float(parts[1]))
        t_s = t_us / 1e6

        g1x = float(parts[5]);  g1y = float(parts[6]);  g1z = float(parts[7])
        g2x = float(parts[11]); g2y = float(parts[12]); g2z = float(parts[13])

        return t_s, g1x, g1y, g1z, g2x, g2y, g2z
    except ValueError:
        return None


def main():
    ser = serial.Serial(PORT, BAUD, timeout=1)
    print(f"Connected to {PORT} @ {BAUD}")

    # Allow ESP reset + any gyro bias calibration phase
    time.sleep(2.0)
    ser.reset_input_buffer()

    smoother = CausalMovingAverage(SMOOTH_WIN)
    fsm = RepStateMachine()

    while True:
        raw = ser.readline()
        if not raw:
            continue

        line = raw.decode(errors="ignore").strip()
        if not line:
            continue

        # Skip header/labels if present
        if line.startswith("SEQ") or "A1X" in line:
            continue

        parsed = parse_line(line)
        if parsed is None:
            continue

        t_s, g1x, g1y, g1z, g2x, g2y, g2z = parsed

        dw_raw = compute_dw_mag_sample(g1x, g1y, g1z, g2x, g2y, g2z)
        dw_s = smoother.update(dw_raw)

        fsm.update(t_s, dw_raw, dw_s)


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\nStopped.")


Connected to COM20 @ 115200
[ 775.888s] REP START (rep=1) dw_s=0.297 >= 0.22 confirm=0.150s dw_raw=0.432
[ 777.118s] REP END   (rep=1, dur=1.23s) dw_s=0.034 <= 0.16 rest=0.260s ROM=18.95deg VALID
            ROM_PROXY = 18.95 deg
[ 782.188s] REP START (rep=2) dw_s=0.347 >= 0.22 confirm=0.160s dw_raw=0.328
[ 782.988s] REP END   (rep=2, dur=0.80s) dw_s=0.084 <= 0.16 rest=0.260s ROM=10.78deg VALID
            ROM_PROXY = 10.78 deg
[ 787.688s] REP START (rep=3) dw_s=0.413 >= 0.22 confirm=0.160s dw_raw=0.374
[ 788.458s] REP END   (rep=3, dur=0.77s) dw_s=0.147 <= 0.16 rest=0.260s ROM=13.32deg VALID
            ROM_PROXY = 13.32 deg
[ 789.918s] REP START (rep=4) dw_s=0.395 >= 0.22 confirm=0.160s dw_raw=0.385
[ 791.998s] REP END   (rep=4, dur=2.08s) dw_s=0.088 <= 0.16 rest=0.250s ROM=29.02deg VALID
            ROM_PROXY = 29.02 deg
[ 792.578s] REP START (rep=5) dw_s=0.494 >= 0.22 confirm=0.160s dw_raw=0.188
[ 796.578s] REP END   (rep=5, dur=4.00s) dw_s=0.044 <= 0.16 rest=0.250s ROM=142.66deg V